In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score,accuracy_score,make_scorer, confusion_matrix, classification_report,accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, average_precision_score, fbeta_score
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_val_score,cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.layers import Dense, Input
from scikeras.wrappers import KerasClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import fbeta_score, make_scorer
from scipy.stats import chi2_contingency,kruskal,kendalltau
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline 
from sklearn.metrics import fbeta_score, make_scorer
from imblearn.pipeline import Pipeline as ImPipeline

### **Split the data into train and test sets**

In [ ]:
imputed_result = pd.read_csv('/Users/xxx/Desktop/LISS data/Cleaned_data.csv')

train_data, test_data = train_test_split(
    imputed_result,
    test_size=0.2,
    random_state=42,
    stratify=imputed_result[['hypertension', 'age_group', 'male']]
)

test_male = test_data[test_data['male'] == 1]
test_female = test_data[test_data['male'] == 0]
test_age_1 = test_data[test_data['age_group'] == 1]
test_age_2 = test_data[test_data['age_group'] == 2]
test_age_3 = test_data[test_data['age_group'] == 3]
test_age_4 = test_data[test_data['age_group'] == 4]
test_age_5 = test_data[test_data['age_group'] == 5]

### **Features sets for training various models** 

In [ ]:
# Model 1 common risk factors

base_variables = [
    'male', 
    'age_group', 
    'BMI', 
    'chest_pain', 
    'angina', 
    'heart_dis', 
    'stroke', 
    'h_cholesterol', 
    'diabetes', 
    'smk'
]

# Model 2 common risk factors + psychological features

variables_2 = base_variables + [
    'anxiety',
    'sleeping_prb', 
    'happiness',
    'life_satisfaction_x'
]   

# Model 3 common risk factors + psychological features + common SES features

variables_3 = base_variables + [
    'anxiety',
    'sleeping_prb', 
    'happiness',
    'life_satisfaction_x',
    'marr_Married',
    'marr_Widow_widower',
    'marr_Never_been_married',
    'marr_Divorced_separated',
    'occu_Is pensioner',
    'occu_No employment',
    'occu_Paid employment',
    'edu_cls'
]   

# Model 4 common risk factors + psychological features + common SES features + incomes + debts

variables_4 = base_variables + [
    'anxiety',
    'sleeping_prb', 
    'happiness',
    'life_satisfaction_x',
    'marr_Married',
    'marr_Widow_widower',
    'marr_Never_been_married',
    'marr_Divorced_separated',
    'occu_Is pensioner',
    'occu_No employment',
    'occu_Paid employment',
    'edu_cls',
    'inc_cls3_Low',
    'inc_cls3_Medium',
    'inc_cls3_High',
    'inc_cls3_Very_High',
    'db_ttl2_cls_None',
    'db_ttl2_cls_Low',
    'db_ttl2_cls_Medium',
    'db_ttl2_cls_High',
    'db_ttl2_cls_Very_High'
]   

### **Logistic regression**

In [ ]:
# Define a function to train the model using the training set

def evaluate_model(data, predictors, target, C, cv_splits=5):
    X = data[predictors]
    y = data[target]
    
    logreg = LogisticRegression(max_iter=100000,C=C)
    cv = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=42)
    
    f2_scorer = make_scorer(fbeta_score, beta=2)
    
    auc_scores = cross_val_score(logreg, X, y, cv=cv, scoring='roc_auc')
    accuracy_scores = cross_val_score(logreg, X, y, cv=cv, scoring='accuracy')
    f1_scores = cross_val_score(logreg, X, y, cv=cv, scoring='f1')
    f2_scores = cross_val_score(logreg, X, y, cv=cv, scoring=f2_scorer)
    recall_scores = cross_val_score(logreg, X, y, cv=cv, scoring='recall')
    precision_scores = cross_val_score(logreg, X, y, cv=cv, scoring='precision')
    
 # Prediction to compute confusion matrix and classification report    
    y_pred = cross_val_predict(logreg, X, y, cv=cv)
    conf_matrix = confusion_matrix(y, y_pred)
    report = classification_report(y, y_pred)
    
    return {
        'Average AUC': np.mean(auc_scores),
        'AUC SD': np.std(auc_scores),
        'Average Accuracy': np.mean(accuracy_scores),
        'Accuracy SD': np.std(accuracy_scores),
        'Average F1 Score': np.mean(f1_scores),
        'F1 Score SD': np.std(f1_scores),
        'Average Recall': np.mean(recall_scores),
        'Recall SD': np.std(recall_scores),
        'Average Precision': np.mean(precision_scores),  # Include average precision score
        'Precision SD': np.std(precision_scores),
        'Confusion Matrix': conf_matrix,
        'Classification Report': report
    }

def print_evaluation_results(results):
    print("Evaluation Results:")
    print(f"Average AUC: {results['Average AUC']:.4f} (SD: {results['AUC SD']:.4f})")
    print(f"Average Accuracy: {results['Average Accuracy']:.4f} (SD: {results['Accuracy SD']:.4f})")
    print(f"Average F1 Score: {results['Average F1 Score']:.4f} (SD: {results['F1 Score SD']:.4f})")
    print(f"Average Recall: {results['Average Recall']:.4f} (SD: {results['Recall SD']:.4f})")
    print(f"Average Precision: {results['Average Precision']:.4f} (SD: {results['Precision SD']:.4f})")  # Print the precision score results
    print("Confusion Matrix:")
    print(results['Confusion Matrix'])
    print("Classification Report:")
    print(results['Classification Report'])

In [ ]:
# Evaluate the model on test set

def evaluate_logistic_regression_on_test(lr, data, predictors, target):

    X_test = data[predictors]
    y_test = data[target]

    # Perform predictions
    y_pred = lr.predict(X_test)
    y_probs = lr.predict_proba(X_test)[:, 1]

    # Calculate metrics
    auc_score = roc_auc_score(y_test, y_probs)
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    confusion = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    # Print the results
    print("Test Evaluation Results:")
    print(f"AUC: {auc_score:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Precision: {precision:.4f}")
    print("Confusion Matrix:")
    print(confusion)
    print("Classification Report:")
    print(report)

In [ ]:
# Grid search for logistic regression for the best C value

def evaluate_model_f1_only(data, predictors, target, cv_splits=5):
    X = data[predictors]
    y = data[target]
    
    logreg = LogisticRegression(max_iter=100000, solver='liblinear')
    param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}  # Range of C values for L2 regularization
    cv = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=42)
    grid_search = GridSearchCV(logreg, param_grid, cv=cv, scoring='f1', refit=True)
    
    grid_search.fit(X, y)
    best_model = grid_search.best_estimator_
    best_f1 = grid_search.best_score_
    
    return {
        'Best C': grid_search.best_params_['C'],
        'Best F1': best_f1
    }

def print_best_f1(results):
    print("Best Model Results:")
    print(f"Best C: {results['Best C']}")
    print(f"Best F1: {results['Best F1']:.4f}")

results = evaluate_model_f1_only(train_data, variables_4, 'hypertension')
print_best_f1(results)

In [ ]:
### Model 1 

results = evaluate_model(train_data,base_variables,'hypertension',C=100)
print_evaluation_results(results)

# test set
lr = LogisticRegression(max_iter=1000, C=100)
X_train = train_data[base_variables]
y_train = train_data['hypertension']
lr.fit(X_train, y_train)

evaluate_logistic_regression_on_test(lr, test_data, base_variables, 'hypertension')

In [ ]:
# test set for male
evaluate_logistic_regression_on_test(lr, test_male, base_variables, 'hypertension')

In [ ]:
# test set for female
evaluate_logistic_regression_on_test(lr, test_female, base_variables, 'hypertension')

In [ ]:
# test set for age gropus
evaluate_logistic_regression_on_test(lr, test_age_1, base_variables, 'hypertension')

In [ ]:
evaluate_logistic_regression_on_test(lr, test_age_2, base_variables, 'hypertension')

In [ ]:
evaluate_logistic_regression_on_test(lr, test_age_3, base_variables, 'hypertension')

In [ ]:
evaluate_logistic_regression_on_test(lr, test_age_4, base_variables, 'hypertension')

In [ ]:
evaluate_logistic_regression_on_test(lr, test_age_5, base_variables, 'hypertension')

In [ ]:
##### Model 2

results2 = evaluate_model(train_data,variables_2,'hypertension',C=10)
print_evaluation_results(results2)

# test set

X_train = train_data[variables_2]
y_train = train_data['hypertension']
lr.fit(X_train, y_train)

evaluate_logistic_regression_on_test(lr, test_data, variables_2, 'hypertension')

In [ ]:
### Model 3

results3 = evaluate_model(train_data,variables_3,'hypertension',C=1)
print_evaluation_results(results3)

# test set

X_train = train_data[variables_3]
y_train = train_data['hypertension']
lr.fit(X_train, y_train)

evaluate_logistic_regression_on_test(lr, test_data, variables_3, 'hypertension')

In [ ]:
### Model 4 

results4 = evaluate_model(train_data,variables_4,'hypertension',C=1)
print_evaluation_results(results4)

# test set

X_train = train_data[variables_4]
y_train = train_data['hypertension']
lr.fit(X_train, y_train)

evaluate_logistic_regression_on_test(lr, test_data, variables_4, 'hypertension')

### **Random forest**

In [ ]:
# Define a function to train the model using the training set

def evaluate_random_forest(data, predictors, target, cv_splits=5, **rf_params):
    
    X = data[predictors]
    y = data[target]

    # Initialize the Random Forest classifier
    rf = RandomForestClassifier(random_state=42, **rf_params)

    # Set up cross-validation
    cv = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=42)

    # Perform cross-validation and calculate metrics
    auc_scores = cross_val_score(rf, X, y, cv=cv, scoring='roc_auc')
    accuracy_scores = cross_val_score(rf, X, y, cv=cv, scoring='accuracy')
    recall_scores = cross_val_score(rf, X, y, cv=cv, scoring='recall')
    precision_scores = cross_val_score(rf, X, y, cv=cv, scoring='precision')
    f1_scores = cross_val_score(rf, X, y, cv=cv, scoring='f1')
    y_pred = cross_val_predict(rf, X, y, cv=cv)

    # Calculate confusion matrix and classification report
    conf_matrix = confusion_matrix(y, y_pred)
    report = classification_report(y, y_pred)

    # Return a dictionary of the results
    return {
        'Average AUC': np.mean(auc_scores),
        'AUC SD': np.std(auc_scores),  
        'Average Accuracy': np.mean(accuracy_scores),
        'Accuracy SD': np.std(accuracy_scores), 
        'Average F1 Score': np.mean(f1_scores),
        'F1 Score SD': np.std(f1_scores),  
        'Average Recall': np.mean(recall_scores),
        'Recall SD': np.std(recall_scores), 
        'Average Precision': np.mean(precision_scores),
        'Precision SD': np.std(precision_scores), 
        'Confusion Matrix': conf_matrix,
        'Classification Report': report
    }

def print_evaluation_results(results):
    print("Evaluation Results:")
    print(f"Average AUC: {results['Average AUC']:.4f} (SD: {results['AUC SD']:.4f})")
    print(f"Average Accuracy: {results['Average Accuracy']:.4f} (SD: {results['Accuracy SD']:.4f})")
    print(f"Average F1 Score: {results['Average F1 Score']:.4f} (SD: {results['F1 Score SD']:.4f})")
    print(f"Average Recall: {results['Average Recall']:.4f} (SD: {results['Recall SD']:.4f})")
    print(f"Average F2 Score: {results['Average F2 Score']:.4f} (SD: {results['F2 Score SD']:.4f})")  # Print F2 score
    print(f"Average Precision: {results['Average Precision']:.4f} (SD: {results['Precision SD']:.4f})")
    print("Confusion Matrix:")
    print(results['Confusion Matrix'])
    print("Classification Report:")
    print(results['Classification Report'])

In [ ]:
# Evaluate the model on test set

def evaluate_random_forest_on_test(rf, data, predictors, target, threshold=0.5):
    
    X_test = data[predictors]
    y_test = data[target]

    # Perform predictions
    y_probs = rf.predict_proba(X_test)[:, 1]  # probabilities for the positive class
    y_pred = (y_probs >= threshold).astype(int)  # apply threshold to generate predictions

    # Calculate metrics
    auc_score = roc_auc_score(y_test, y_probs)
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    f2 = fbeta_score(y_test, y_pred, beta=2) 
    precision = precision_score(y_test, y_pred)
    confusion = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    print("Test Evaluation Results:")
    print(f"AUC: {auc_score:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"F2 Score: {f2:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Precision: {precision:.4f}")
    print("Confusion Matrix:")
    print(confusion)
    print("Classification Report:")
    print(report)

In [ ]:
# Optuna for optimizing random forest parameters

def objective(trial, data, predictors, target, cv_splits=5):
    X = data[predictors]
    y = data[target]

    # Define hyperparameters to be tuned

    n_estimators = trial.suggest_int('n_estimators', 50, 500, step=50)
    max_depth = trial.suggest_int('max_depth', 3, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    class_weight = trial.suggest_categorical('class_weight', ['balanced'])
    bootstrap = trial.suggest_categorical('bootstrap', [False])
    
    # Initialize the Random Forest classifier with the suggested parameters
    rf = RandomForestClassifier(
        random_state=42,
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        class_weight=class_weight,
        bootstrap=bootstrap
    )

    # Set up cross-validation
    cv = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=42)

    # # Best F1
    f1_scores = cross_val_score(rf, X, y, cv=cv, scoring='f1')
    return np.mean(f1_scores)

# Run Optuna optimization
def run_optuna(data, predictors, target, n_trials=50, cv_splits=5):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, data, predictors, target, cv_splits), n_trials=n_trials,show_progress_bar=False)
    return study

# Example usage:
# study = run_optuna(train_data,base_variables,'hypertension',n_trials=500)
# print(study.best_value)
# study.best_params

In [ ]:
# Identify the optimal threshold balancing precision and recall

def plot_precision_recall_vs_threshold(model, data, predictors, target):

    X_test = data[predictors]
    y_test = data[target]

    # Getting the probability scores of the positive class
    y_scores = model.predict_proba(X_test)[:, 1]
    
    # Calculate precision, recall, and thresholds
    precision, recall, thresholds = precision_recall_curve(y_test, y_scores)
    
    # Calculate F1 score for each threshold
    f1_scores = np.where((precision + recall) == 0, 0, 2*(precision*recall)/(precision+recall))

    
    # Identify the threshold that gives the best F1 score
    best_threshold = thresholds[np.argmax(f1_scores)]
    best_f1_score = np.max(f1_scores)
    
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(thresholds, precision[:-1], "b--", label="Precision")
    plt.plot(thresholds, recall[:-1], "g-", label="Recall")
    plt.plot(thresholds, f1_scores[:-1], "r-", label="F1 Score")
    plt.title("Precision, Recall, and F1 Score for different thresholds")
    plt.xlabel("Threshold")
    plt.ylabel("Score")
    plt.legend(loc="best")
    plt.grid(True)
    plt.axvline(x=best_threshold, color='gray', linestyle='--', label=f'Best Threshold: {best_threshold:.4f}')
    plt.show()

    print(f"Best F1 Score: {best_f1_score:.4f} at Threshold: {best_threshold:.4f}")

# plot_precision_recall_vs_threshold(rf, test_data, variables_4, 'hypertension')

In [ ]:
study = run_optuna(train_data,base_variables,'hypertension',n_trials=500)

In [ ]:
print(study.best_value)
study.best_params

In [ ]:
# Model 1 with Optuna-tuned parameters, 'class_weight'='balanced' 

best_params = {'n_estimators': 100,
 'max_depth': 10,
 'min_samples_split': 16,
 'min_samples_leaf': 10,
 'max_features': 'sqrt',
 'class_weight': 'balanced',
 'bootstrap': True}


results = evaluate_random_forest(train_data,base_variables,'hypertension',**best_params)
print_evaluation_results(results)

# # Test set

rf = RandomForestClassifier(random_state=42, **best_params)
X_train = train_data[base_variables]
y_train = train_data['hypertension']
rf.fit(X_train, y_train) 
evaluate_random_forest_on_test(rf, test_data, base_variables, 'hypertension')

In [ ]:
plot_precision_recall_vs_threshold(rf, test_data, base_variables, 'hypertension')

In [ ]:
evaluate_random_forest_on_test(rf, test_data, base_variables, 'hypertension',threshold=0.5807)

In [ ]:
study = run_optuna(train_data,variables_2,'hypertension',n_trials=500)

In [ ]:
print(study.best_value)
study.best_params

In [ ]:
# Model 2 with Optuna-tuned parameters, 'class_weight'='balanced' 

best_params = {'n_estimators': 100,
 'max_depth': 18,
 'min_samples_split': 19,
 'min_samples_leaf': 7,
 'max_features': 'log2',
 'class_weight': 'balanced',
 'bootstrap': False}

results2 = evaluate_random_forest(train_data,variables_2,'hypertension',**best_params)
print_evaluation_results(results2)

# # Test set

rf = RandomForestClassifier(random_state=42, **best_params)
X_train = train_data[variables_2]
y_train = train_data['hypertension']
rf.fit(X_train, y_train) 
evaluate_random_forest_on_test(rf, test_data, variables_2, 'hypertension',threshold = 0.5)

In [ ]:
plot_precision_recall_vs_threshold(rf, test_data, variables_2, 'hypertension')

In [ ]:
evaluate_random_forest_on_test(rf, test_data, variables_2, 'hypertension',threshold = 0.5551)

In [ ]:
# Male
evaluate_random_forest_on_test(rf, test_male, variables_2, 'hypertension',threshold = 0.5)

In [ ]:
# Female
evaluate_random_forest_on_test(rf, test_female, variables_2, 'hypertension',threshold = 0.5)

In [ ]:
# Age groups
evaluate_random_forest_on_test(rf, test_age_1, variables_2, 'hypertension',threshold = 0.5)

In [ ]:
evaluate_random_forest_on_test(rf, test_age_2, variables_2, 'hypertension',threshold = 0.5)

In [ ]:
evaluate_random_forest_on_test(rf, test_age_3, variables_2, 'hypertension',threshold = 0.5)

In [ ]:
evaluate_random_forest_on_test(rf, test_age_4, variables_2, 'hypertension',threshold = 0.5)

In [ ]:
evaluate_random_forest_on_test(rf, test_age_5, variables_2, 'hypertension',threshold = 0.5)

In [ ]:
study = run_optuna(train_data,variables_3,'hypertension',n_trials=500)

In [ ]:
print(study.best_value)
study.best_params

In [ ]:
# Model 3 with Optuna-tuned parameters, 'class_weight'='balanced' 

best_params = {'n_estimators': 450,
 'max_depth': 17,
 'min_samples_split': 19,
 'min_samples_leaf': 7,
 'max_features': 'log2',
 'class_weight': 'balanced',
 'bootstrap': False}


results3 = evaluate_random_forest(train_data,variables_3,'hypertension',**best_params)
print_evaluation_results(results3)

# # Test set

rf = RandomForestClassifier(random_state=42, **best_params)
X_train = train_data[variables_3]
y_train = train_data['hypertension']
rf.fit(X_train, y_train) 
evaluate_random_forest_on_test(rf, test_data, variables_3, 'hypertension')

In [ ]:
plot_precision_recall_vs_threshold(rf, test_data, variables_3, 'hypertension')

In [ ]:
evaluate_random_forest_on_test(rf, test_data, variables_3, 'hypertension', threshold=0.5207)

In [ ]:
# Male
evaluate_random_forest_on_test(rf, test_male, variables_3, 'hypertension',threshold = 0.5207)

In [ ]:
# Female
evaluate_random_forest_on_test(rf, test_female, variables_3, 'hypertension',threshold = 0.5207)

In [ ]:
evaluate_random_forest_on_test(rf, test_age_1, variables_3, 'hypertension',threshold = 0.5207)

In [ ]:
evaluate_random_forest_on_test(rf, test_age_2, variables_3, 'hypertension',threshold = 0.5207)

In [ ]:
evaluate_random_forest_on_test(rf, test_age_3, variables_3, 'hypertension',threshold = 0.5207)

In [ ]:
evaluate_random_forest_on_test(rf, test_age_4, variables_3, 'hypertension',threshold = 0.5207)

In [ ]:
evaluate_random_forest_on_test(rf, test_age_5, variables_3, 'hypertension',threshold = 0.5207)

In [ ]:
study = run_optuna(train_data,variables_4,'hypertension',n_trials=500)

In [ ]:
print(study.best_value)
study.best_params

In [ ]:
# Model 4 with Optuna-tuned parameters, 'class_weight'='balanced' 

best_params = {'n_estimators': 50,
 'max_depth': 14,
 'min_samples_split': 15,
 'min_samples_leaf': 9,
 'max_features': 'log2',
 'class_weight': 'balanced',
 'bootstrap': False}

results4 = evaluate_random_forest(train_data,variables_4,'hypertension',**best_params)
print_evaluation_results(results4)

# # Test set

rf = RandomForestClassifier(random_state=42, **best_params)
X_train = train_data[variables_4]
y_train = train_data['hypertension']
rf.fit(X_train, y_train) 
evaluate_random_forest_on_test(rf, test_data, variables_4, 'hypertension')

In [ ]:
plot_precision_recall_vs_threshold(rf, test_data, variables_4, 'hypertension')

In [ ]:
evaluate_random_forest_on_test(rf, test_data, variables_4, 'hypertension', threshold=0.4364)

### **Random forest with SMOTE**

In [ ]:
# Define a function to train the model using the training set

def evaluate_random_forest(data, predictors, target, cv_splits=5, k_neighbors=5, **best_params):
    X = data[predictors]
    y = data[target]

    # Initialize SMOTE and Random Forest classifier
    smote = SMOTE(random_state=42, k_neighbors=k_neighbors)
    rf = RandomForestClassifier(random_state=42, **best_params)

    # Create a pipeline including SMOTE and Random Forest
    pipeline = Pipeline([
        ('smote', smote),
        ('random_forest', rf)
    ])

    # Set up stratified cross-validation
    cv = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=42)

    # Perform cross-validation and calculate metrics
    auc_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='roc_auc')
    accuracy_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')
    recall_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='recall')
    precision_scores = cross_val_score(rf, X, y, cv=cv, scoring='precision')
    f1_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='f1')
    y_pred = cross_val_predict(pipeline, X, y, cv=cv)

    # Calculate confusion matrix and classification report
    conf_matrix = confusion_matrix(y, y_pred)
    report = classification_report(y, y_pred)

    # Return a dictionary of the results
    return {
        'Average AUC': np.mean(auc_scores),
        'AUC SD': np.std(auc_scores),
        'Average Accuracy': np.mean(accuracy_scores),
        'Accuracy SD': np.std(accuracy_scores),
        'Average F1 Score': np.mean(f1_scores),
        'F1 Score SD': np.std(f1_scores),
        'Average Recall': np.mean(recall_scores),
        'Recall SD': np.std(recall_scores),
        'Average Precision': np.mean(precision_scores),
        'Precision SD': np.std(precision_scores), 
        'Confusion Matrix': conf_matrix,
        'Classification Report': report
    }

def print_evaluation_results(results):
    print("Evaluation Results:")
    print(f"Average AUC: {results['Average AUC']:.4f} (SD: {results['AUC SD']:.4f})")
    print(f"Average Accuracy: {results['Average Accuracy']:.4f} (SD: {results['Accuracy SD']:.4f})")
    print(f"Average F1 Score: {results['Average F1 Score']:.4f} (SD: {results['F1 Score SD']:.4f})")
    print(f"Average Recall: {results['Average Recall']:.4f} (SD: {results['Recall SD']:.4f})")
    print(f"Average Precision: {results['Average Precision']:.4f} (SD: {results['Precision SD']:.4f})")
    print("Confusion Matrix:")
    print(results['Confusion Matrix'])
    print("Classification Report:")
    print(results['Classification Report'])

In [ ]:
# Evaluate random forest model on test set

def evaluate_random_forest_on_test(rf, data, predictors, target, threshold=0.5):

    X_test = data[predictors]
    y_test = data[target]

    # Perform predictions
    y_pred = rf.predict(X_test)
    y_probs = rf.predict_proba(X_test)[:, 1]

    # Calculate metrics
    auc_score = roc_auc_score(y_test, y_probs)
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    f2 = fbeta_score(y_test, y_pred, beta=2)
    confusion = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    # Print the results
    print("Test Evaluation Results:")
    print(f"AUC: {auc_score:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"F2 Score: {f2:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Precision: {precision:.4f}")
    print("Confusion Matrix:")
    print(confusion)
    print("Classification Report:")
    print(report)

In [ ]:
# Optuna for optimizing random forest parameters

def objective(trial, data, predictors, target, cv_splits=5):
    X = data[predictors]
    y = data[target]
    # Define and suggest hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 500, step=50)
    max_depth = trial.suggest_int('max_depth', 3, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    class_weight = trial.suggest_categorical('class_weight', [None])
    bootstrap = trial.suggest_categorical('bootstrap', [False])
    smote_neighbors = trial.suggest_int('smote_neighbors', 3, 9)
    # Setup the pipeline
    pipeline = ImPipeline([
        ('smote', SMOTE(k_neighbors=smote_neighbors)),
        ('classifier', RandomForestClassifier(
            random_state=42,
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            class_weight=class_weight,
            bootstrap=bootstrap
        ))
    ])
    # Cross-validation
    cv = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=42)
    f1_scorer = make_scorer(f1_score)
    f1_scores = cross_val_score(pipeline, X, y, cv=cv, scoring=f1_scorer, error_score='raise')
    return np.mean(f1_scores)

# Running the study
def run_optuna(data, predictors, target, n_trials=50, cv_splits=5):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, data, predictors, target, cv_splits), n_trials=n_trials)
    return study

# study = run_optuna(train_data, base_variables, 'hypertension', n_trials=500)
# best_params = study.best_trial.params
# best_params

In [ ]:
study = run_optuna(train_data, base_variables, 'hypertension', n_trials=500)

In [ ]:
best_params = study.best_trial.params
best_params

In [ ]:
# Model 1 with Optuna-tuned parameters, 'class_weight'='none' 

best_params = {'n_estimators': 150,
 'max_depth': 3,
 'min_samples_split': 3,
 'min_samples_leaf': 9,
 'max_features': 'log2',
 'class_weight': None,
 'bootstrap': False
              }

results = evaluate_random_forest(train_data,base_variables,'hypertension',cv_splits=8, k_neighbors=6,**best_params)
print_evaluation_results(results)

rf = RandomForestClassifier(random_state=42, **best_params)
X_train = train_data[base_variables]
y_train = train_data['hypertension']
rf.fit(X_train, y_train) 
evaluate_random_forest_on_test(rf, test_data, base_variables, 'hypertension')

In [ ]:
study = run_optuna(train_data, variables_2, 'hypertension', n_trials=500)

In [ ]:
best_params2 = study.best_trial.params
best_params2

In [ ]:
# Model 2 with Optuna-tuned parameters, 'class_weight'='none' 

best_params = {'n_estimators': 200,
 'max_depth': 10,
 'min_samples_split': 14,
 'min_samples_leaf': 9,
 'max_features': 'log2',
 'class_weight': None,
 'bootstrap': False
              }

results = evaluate_random_forest(train_data,variables_2,'hypertension',cv_splits=5, k_neighbors=3,**best_params)
print_evaluation_results(results)

rf = RandomForestClassifier(random_state=42, **best_params)
X_train = train_data[variables_2]
y_train = train_data['hypertension']
rf.fit(X_train, y_train) 
evaluate_random_forest_on_test(rf, test_data, variables_2, 'hypertension')

In [ ]:
study = run_optuna(train_data, variables_3, 'hypertension', n_trials=500)

In [ ]:
best_params3 = study.best_trial.params
best_params3

In [ ]:
# Model 3 with Optuna-tuned parameters, 'class_weight'='none' 

best_params = {'n_estimators': 100,
 'max_depth': 4,
 'min_samples_split': 10,
 'min_samples_leaf': 10,
 'max_features': 'sqrt',
 'class_weight': None,
 'bootstrap': False
              }

results = evaluate_random_forest(train_data,variables_3,'hypertension',cv_splits=5, k_neighbors=6,**best_params)
print_evaluation_results(results)

rf = RandomForestClassifier(random_state=42, **best_params)
X_train = train_data[variables_3]
y_train = train_data['hypertension']
rf.fit(X_train, y_train) 
evaluate_random_forest_on_test(rf, test_data, variables_3, 'hypertension')

In [ ]:
study = run_optuna(train_data, variables_4, 'hypertension', n_trials=500)

In [ ]:
best_params4 = study.best_trial.params
best_params4

In [ ]:
# Model 4 with Optuna-tuned parameters, 'class_weight'='none' 

best_params = {'n_estimators': 50,
 'max_depth': 4,
 'min_samples_split': 16,
 'min_samples_leaf': 8,
 'max_features': 'sqrt',
 'class_weight': None,
 'bootstrap': False
              }

results = evaluate_random_forest(train_data,variables_4,'hypertension',cv_splits=3, k_neighbors=3,**best_params)
print_evaluation_results(results)

rf = RandomForestClassifier(random_state=42, **best_params)
X_train = train_data[variables_4]
y_train = train_data['hypertension']
rf.fit(X_train, y_train) 
evaluate_random_forest_on_test(rf, test_data, variables_4, 'hypertension')